In [6]:
!pip install fasttext

In [7]:
!pip install nltk

In [8]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [9]:
# importing libraries
import pandas as pd
import numpy as np
import re

In [10]:
# Load the dataset
df = pd.read_csv('../Datasets/fsa_prod_dump.csv', on_bad_lines='skip')
df.head()

,L0_Category,L1_Class,L2_Category,L3_Dictionary_SubCategory,L4_dictionarySubgroup,ProdName,ItemCode,Vendor,ProdType
0,NaN,NaN,NaN,NaN,NaN,GROUND CHUCK FLAP 50% CHUCK NECK 50%,623,Golden Gate Meat Company,Canonical
1,Food,Food - Meat & Game,Beef -- Ground & Shapes,Beef (Bulk) - Ground,Beef Ground >10% Fat (NAMP 136),FINE GRIND BEEF TUBES 81/19 6/10#,624,Golden Gate Meat Company,Canonical
2,Food,Food - Meat & Game,Beef - Foodservice Whole Cuts,Beef (Bulk) - Plate Cuts,Outside Skirt - Skinned (NAMP # 121E),AUST OUTSIDE SKIRT PEELED SOLD BY CASE,1717,Golden Gate Meat Company,Canonical
3,Food,Food - Meat & Game,Beef Deli & Charcuterie,Beef - Deli & Charcuterie,Beef Pastrami (NAMP # 611),PASTRAMI - NEW YORK STYLE- SABRA,1820,Golden Gate Meat Company,Canonical
4,Food,Food - Meat & Game,Beef Deli & Charcuterie,Beef - Deli & Charcuterie,Beef Pastrami (NAMP # 611),PASTRAMI SLICED 5# BAGS (50# CASE),1821,Golden Gate Meat Company,Canonical


In [11]:
df.shape

(1021560, 9)

In [12]:
# Droppig missing values
df = df.dropna(axis=0)

# Drop duplicates
df.drop_duplicates(keep='first', inplace=True)

# **L2 Category**

In [13]:
# Selecting columns for L1 category
columns = ['L0_Category', 'L1_Class', 'L2_Category', 'ProdName']

df = df[columns]
df.head(3)

,L0_Category,L1_Class,L2_Category,ProdName
1,Food,Food - Meat & Game,Beef -- Ground & Shapes,FINE GRIND BEEF TUBES 81/19 6/10#
2,Food,Food - Meat & Game,Beef - Foodservice Whole Cuts,AUST OUTSIDE SKIRT PEELED SOLD BY CASE
3,Food,Food - Meat & Game,Beef Deli & Charcuterie,PASTRAMI - NEW YORK STYLE- SABRA


In [14]:
df.L2_Category.unique()

array(['Beef -- Ground & Shapes', 'Beef - Foodservice Whole Cuts',
       'Beef Deli & Charcuterie', 'Savory Sauces',
       'Pork -- Foodservice Whole Cuts', 'Snack Kits or Combos',
       'Pork -- Bones - Offal - Extremities - Fat',
       'Lamb - Foodservice Whole Cuts', 'Pork -- Bacon',
       'Turkey -- Value Added', 'Chicken -- Value Added',
       'Dairy -- Cheese', 'Turkey -- Ground & Sub-Part Pieces',
       'Duck -- Foodservice Cuts',
       'Duck -- Bones - Offal - Extremities - Fat', 'Fowl - Pheasant Etc',
       'Venison', 'Wild Boar', 'Shellfish -- Shrimp & Prawn',
       'Shellfish -- Lobster - Crabs - Seashell',
       'Shellfish -- All BiValves',
       'Beef -- Portioned Incl Steaks & Chops',
       'Admin -- Unclassifiable Products',
       'Beef Bones - Offal - Extremities - Fat',
       'Veal -- Foodservice Whole Cuts',
       'Veal -- Foodservice Portion Cuts',
       'Veal -- Bones - Offal - Fat - Extremities',
       'Pork -- Portioned Incl Steaks & Chops', 'Por

In [15]:
# Counting L2 Category classes
len(df.L2_Category.unique())

280

In [16]:
# Importing Libraries
# import nltk
# from nltk.corpus import stopwords

# # Download stopwords
# nltk.download("stopwords")

In [17]:
# Preprocessing fuction for category
def class_pre(text):
  new_text = re.sub(r'&', 'and', text)
  new_text = re.sub(r'[^\w\s]',' ', new_text)
  new_text = re.sub(r' +', ' ', new_text)
  new_text = re.sub(r'\s', '_', new_text)
  return new_text

In [18]:
# Did a slight modification, convert & into and
# Then removes extra whitespaces

# def clean(string):
#     string = re.sub('&', 'and ', string)
#     raw_text = re.sub("[^a-zA-Z]+", " ", string)
#     raw_text = re.sub(' +', ' ', raw_text)
#     raw_text = raw_text.strip()
#     words = raw_text.lower().split()
#     stops = set(stopwords.words("english"))
#     meaningful_words = [
#         word for word in words if ((not word in stops) and (len(word) >= 3))
#     ]
#     string = " ".join(meaningful_words)
#     return string

In [19]:
# uniques = df.L2_Category.unique()

# for word in uniques:
#   pp = class_pre(word)
#   print(pp)

In [20]:
# making the category
df['L2'] = df['L2_Category'].apply(class_pre)

In [21]:
# Creating label of the class according to the fasttext format
df['category'] = '__label__' + df['L2'].astype(str)

In [22]:
# Drop used columns
df = df.drop(['L2_Category', 'L2'], axis=1)

In [23]:
# Creating catehory description with producbt name and vendor
df['category_description'] = df['category'] + ' ' + df['L1_Class'] + ' ' + df['ProdName']
# df['category_description'] = df['L0_Category'] + ' ' + df['L1_Class'] + ' ' + df['ProdName']

In [24]:
#  Drop the columns : ProdName, L0_Category, L1_Class
df = df.drop(['ProdName', 'L0_Category', 'L1_Class'], axis=1)

In [25]:
df.head()

,category,category_description
1,__label__Beef_Ground_and_Shapes,__label__Beef_Ground_and_Shapes Food - Meat & ...
2,__label__Beef_Foodservice_Whole_Cuts,__label__Beef_Foodservice_Whole_Cuts Food - Me...
3,__label__Beef_Deli_and_Charcuterie,__label__Beef_Deli_and_Charcuterie Food - Meat...
4,__label__Beef_Deli_and_Charcuterie,__label__Beef_Deli_and_Charcuterie Food - Meat...
5,__label__Beef_Deli_and_Charcuterie,__label__Beef_Deli_and_Charcuterie Food - Meat...


In [26]:
# Replace all white spaces whith single space
# Remove white spaces in begining and end of the text
# Turn text into lower case

# Preprocessing Function
def preprocess(text):
    text = re.sub(r'&', 'and', text)
    text = re.sub(r'[^\w\s]',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [27]:
# Apply preprocessing function to 'category_description'

df['category_description'] = df['category_description'].apply(preprocess)
# df['category_description'] = df['category'].apply(preprocess) + ' ' + df['category_description'].apply(clean)

In [29]:
# Test Train splitting
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.02, random_state=42)

In [30]:
# Save test and train dataset into two differnet files
train.to_csv('train_set.train',
             columns=["category_description"],
             index=False,
             header=False)

test.to_csv('test_set.train',
             columns=["category_description"],
             index=False,
             header=False)

In [31]:
# Fast text model
import fasttext

model = fasttext.train_supervised(input='train_set.train',
                                  dim=600,
                                  loss='hs',
                                  wordNgrams=3,
                                  lr=0.35,
                                  epoch=15,
                                  minn=3,
                                  maxn=5)

model.test('test_set.train')

(13917, 0.9586836243443271, 0.9586836243443271)

In [32]:
text = 'Operationals - Service (FOH) Supplies Bag Paper Kraft wine large'
text = preprocess(text)
model.predict(text, k=5)
# True L2 Class = Bags - Service - Disposable

(('__label__bags_service_disposable',
  '__label__containers_for_service_disposable',
  '__label__carriers_and_trays_service_disposable',
  '__label__service_wares_tableware',
  '__label__dairy_milk_and_cream'),
 array([9.99511898e-01, 3.30449519e-04, 1.77310591e-04, 4.06580621e-05,
        2.72543402e-05]))

In [33]:
model.save_model("../Models/L2/tuned_model.bin")